This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [82]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [83]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [84]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://data.nasdaq.com/api/v3/' + 'datasets/FSE/AFX_X' + '/data.json?' + API_KEY)
r.status_code

200

In [85]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = r.json()
json_data['dataset_data']['column_names']
#print(json_data)

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [86]:
AFX_X_2017 = []

#subset the json dictionary to only have stock market data
stock_data = json_data['dataset_data']['data']


#subset data to only contain data from '2017' by checking the first 5 chars for the year
#if the data is 2017, appends the row of data to AFX_X_2017 to be coverted to a dict later
for stock in stock_data:
    for data in stock:
        if str(data)[:5] == '2017-':
            #print(data)
            AFX_X_2017.append(stock)

            
#creates a dictionary for the json data contained in AFX_X_2017
#key is a number starting from 1 (and increments by 1) and the values are the columns of data
AFX_X_2017_dict = dict(enumerate(AFX_X_2017))

In [87]:
#for k, v in AFX_X_2017_dict.items():
#    print(v)

In [88]:
#highest_opening = max([v for k, v in AFX_X_2017_dict.items() if AFX_X_2017_dict[k][1] is not None])
#lowest_opening = min([v for k, v in AFX_X_2017_dict.items() if AFX_X_2017_dict[k][1] is not None])


#initialize variables to hold the index (the key) of the desired stat
#the variables are initalized to the first row of data as a basis of comparsion
highest_key, highest_open = 0, AFX_X_2017[0][1]                             #highest opening day
lowest_key, lowest_open = 0, AFX_X_2017[0][1]                               #lowest opening dat
change1_key, largest_change1 = 0, AFX_X_2017[0][2] - AFX_X_2017[0][3]       #largest change in one day 
change2_key, largest_change2 = 0, AFX_X_2017[0][4]                          #largest change in two days
volume_sum = 0                                                              #tracks sums all of the daily trade volumes
counter = 0                                                                 #used to divide 'volume_sum' to get average
median_trade_list = []                                                      #stores all of daily trade volume



#iterates through all of the dictionary
for k, v in AFX_X_2017_dict.items():
    
    #checks if the value is not Null, originally we could have filtered out all null
    #data but just because one column is missing data does not neccesarily mean other
    #columns are missing data relevant to the assignment, so I decided to check each column
    #individually for missing data instead
    
    
    #checks first column, the opening price
    if v[1] is not None:
        
        
        #does a comparsion if the current opening price is higher than the value stored in
        #'highest_open', if so the current opening price is the new highest price and we
        #store the index because having the date is useful
        
        if v[1] > highest_open:
            highest_open = v[1]
            highest_key = k
        
        
        #does a comparsion if the current opening price is lowerthan the value stored in
        #'lowest_open', if so the current opening price is the new highest price, updates
        #the index 
        
        if v[1] < lowest_open:
            lowest_open = v[1]
            lowest_key = k
     
    
    #checks columns for the highest and lowest price, subtracts the two value to determine
    #the difference of the stock in day, updates 'largest_change1' is the current difference
    #is greater than the one stored in 'largest_change1'
    if v[2] and v[3] is not None:
        
        if v[2]-v[3] > largest_change1:
            largest_change1 = v[2] - v[3]
            change1_key = k
    
    
    #checks column for the largest change in two days, performs comparsion similarly to the
    #previous steps
    if v[4] is not None:
        
        if v[4] > largest_change2:
            largest_change2 = v[4]
            change2_key = k
    
    
    #checks column for the daily trade volume, sums up the volume and store it in 'volume_sum'
    #increments 'counter' so that we have something to calculate the average with later on
    #appends the volume to 'median_trade_list' to determine median trade volume
    if v[6] is not None:
        volume_sum += v[6]
        counter += 1
        median_trade_list.append(v[6])

In [89]:
#sorts the trade volume from lowest to highest
median_trade_list.sort()

#gets the median trade volume, integer divide the length of the list by 2 but subtract 1 first to
#account for lists starting at 0, regular division may return a float if the list is of odd length
#so integer division is the beter option
median_trade_volume = median_trade_list[int((len(median_trade_list) - 1)//2)]

In [90]:
print("The highest opening was on", AFX_X_2017[highest_key][0], "at $", highest_open)
print("The lowest opening was on", AFX_X_2017[lowest_key][0], "at $", lowest_open)
print("The largest change in any one day was on", AFX_X_2017[change1_key][0], "with $",largest_change1)
print("the largest change between any two days was on", AFX_X_2017[change2_key][0], "with $", largest_change2)
print("The average daily trading volume during 2017 was", volume_sum/counter)
print("The median trade volume during 2017 was", median_trade_volume)

The highest opening was on 2017-12-14 at $ 53.11
The lowest opening was on 2017-01-24 at $ 34.0
The largest change in any one day was on 2017-05-11 with $ 2.8100000000000023
the largest change between any two days was on 2017-12-13 with $ 53.09
The average daily trading volume during 2017 was 89124.33725490196
THe median trade volume during 2017 was 76286.0
